In [1]:
%run ./PrimeNumber.ipynb

In [2]:
### Genric DLP Algorithm
# G = (op, e)
# op: group operation
# e: identity
# a: element of G
# n: some positive integer such that a^n = e

def Generic_PositiveExp(a, x, op, e):
    assert x >= 0
    b = e
    z = 1
    c = a
    while z <= x:
        if x & z: b = op(b, c)
        c = op(c, c)
        z = z << 1
    return b

def Generic_Inv(a, op, e, n):
    assert Generic_PositiveExp(a, n, op, e) == e
    return Generic_PositiveExp(a, n-1, op, e)

def Generic_Exp(a, x, op, e, n=None):
    if x >= 0:
        return Generic_PositiveExp(a, x, op, e)
    else:
        assert n != None and n > 0
        a = Generic_Inv(a, op, e, n)
        return Generic_PositiveExp(a, -x, op, e)
    
def Generic_BabyStepGiantStep(a, b, op, e, n):
    #assert n > 0 and Generic_Exp(a, n, op, e) == e
    x = GreatestSquareRoot(n) + 1
    L = list()
    c = one
    for i in range(0, x):
        L.append((i, c))
        c = op(c, a)
    L = sorted(L, key=lambda item: item[1]);# print(L)
    c = b
    for i in range(0, x):
        k = BinarySearch(0, x-1, lambda j: L[j][1] <= c)
        if k != None and L[k][1] == c:
            return i*x+L[k][0]
        c = op(c, Generic_Exp(a, -x, op, e, n))
    return None

def Generic_PohligHellman(a, b, op, e, n):
    #assert n > 0 and Generic_Exp(a, n, op, e) == e
    divisors = SmoothFactoring(n)
    m = 1
    for divisor in divisors:
        m *= divisor[0]**divisor[1]
    if m < n:
        divisors.append((n//m, 1))
    #print(divisors)
    congruences = list()
    for divisor in divisors:
        w = 0
        y = divisor[0]**divisor[1]
        z = n//y
        s_0 = Generic_Exp(a, z, op, e)
        t = Generic_Exp(b, z, op, e)
        s_1 = Generic_Exp(a, z*(divisor[0]**(divisor[1]-1)), op, e)
        for i in range(0, divisor[1]):
            u = Generic_Exp(s_0, -w, op, e, y)
            t_0 = op(t, u)
            t_1 = Generic_Exp(t_0, divisor[0]**(divisor[1]-i-1), op, e)
            j = Generic_BabyStepGiantStep(s_1, t_1, op, e, divisor[0])
            w += j*(divisor[0]**i)
        congruences.append((y, w))
    #print(congruences)
    (m, x) = MultipleCRT(congruences)
    return x

def Generic_DiscreteLog(a, b, op, e, n):
    return Generic_PohligHellman(a, b, op, e, n)